In [1]:
import ftplib 
from ftplib import FTP
import subprocess
import pip._internal.commands as commands
import os
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from os import listdir
from os.path import isfile, join
import pyodbc
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [7]:
HOSTNAME = 'ftp.datasus.gov.br'
ftp_server = ftplib.FTP(HOSTNAME)
ftp_server.login() 
ftp_server.cwd('dissemin/publicos/SINAN/DADOS/FINAIS/')
filename = "LEIVBR07.dbc"
with open(filename, "wb") as file:    
    ftp_server.retrbinary(f"RETR {filename}", file.write) 

In [10]:
def dbccsv(raw_filename):
    raw_files_dir="In_Out_Files/"
    converted_files_dir="In_Out_Files/"
    path2script= "dbc_csv.R"
    arg = '--vanilla'
    dbccsv_path = "dbc_csv.R"+ raw_files_dir+ " " + converted_files_dir + " " + raw_filename

    try:
        r_script_path = commands.getstatusoutput('which Rscript')[1]
        subprocess.call(['C:/Program Files/R/R-4.2.1/bin/Rscript.exe',path2script,raw_files_dir,raw_files_dir,raw_filename], shell=True)
        return True
    except:
        print("(Rscript) Error converting file: " + raw_filename)

    return False

In [2]:
def dbc2csv(raw_filename):
   command = 'C:/Program Files/R/R-4.2.1/bin/Rscript.exe' 
   #command = 'Rscript'                    # OR WITH bin FOLDER IN PATH ENV VAR 
   arg = '--vanilla' 

   try: 
     p = subprocess.Popen([command, arg,
                          "dbc_csv.R",
                          "In_Out_Files/", 
                          "In_Out_Files/",   
                          raw_filename],
                          stdin = subprocess.PIPE, 
                          stdout = subprocess.PIPE, 
                          stderr = subprocess.PIPE) 

     output, error = p.communicate() 

     if p.returncode == 0: 
        i=1 
     else: 
        print('R ERROR:\n {0}'.format(error.decode("utf-8"))) 

     return True

   except Exception as e: 
     print("dbc2csv - Error converting file: " + raw_filename) 
     print(e)


     return False

In [3]:
path = 'In_Out_Files'
files = [f for f in listdir(path) if isfile(join(path, f))]

In [13]:
for item in files:
    dbc2csv(item)

In [2]:
DBFile = r'BD/EstudoLeishmaniose.accdb'
path = 'Out'
files = [f for f in listdir(path) if isfile(join(path, f))]

for item in files:
      exFile = r'Out/'+item
      conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+DBFile)
      
      # APPEND TO LOCAL TABLE
      sql = """INSERT INTO TB_DataSUS_Tergumentar 
            SELECT * FROM [Excel 12.0 Xml; HDR = Yes;Database={myfile}].[Sheet 1$];
            """
      curs = conn.cursor()
      curs.execute(sql.format(myfile = exFile))
      conn.commit()

Error: ('21S01', '[21S01] [Microsoft][Driver ODBC Microsoft Access] Número de valores da consulta e campos de destino não coincidem. (-3520) (SQLExecDirectW)')